In [8]:
import pandas as pd
import numpy as np
import pickle

import pypyodbc as podbc
from pyvis.network import Network


import warnings
warnings.filterwarnings("ignore", category=UserWarning)




In [2]:
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'NicksComputer\SQLEXPRESS'
DATABASE_NAME = 'movie_recsys'

connection_string = f"""
DRIVER={{{DRIVER_NAME}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
Trust_Connection=yes;
"""

conn = podbc.connect(connection_string)

## Print top 5 rows from each table

In [3]:
sql = []
table_str = []

table_str.append('Name Basics')
table_str.append('Title Basics')
table_str.append('Title Crew')
table_str.append('Title Episodes')
table_str.append('Title Principals')
table_str.append('Title Ratings')
sql.append("""
           SELECT TOP(5) *
           FROM nameBasics 
           """)
sql.append("""
           SELECT TOP(5) *
           FROM titleBasics 
           """)
sql.append("""
           SELECT TOP(5) *
           FROM titleCrew
           """)
sql.append("""
           SELECT TOP(5) *
           FROM titleEpisode
           """)
sql.append("""
           SELECT TOP(5) *
           FROM titlePrincipals 
           """)
sql.append("""
           SELECT TOP(5) *
           FROM titleRatings
           """)

for j, jsql in enumerate(sql):
    print(table_str[j])
    print(pd.read_sql_query(jsql, conn).to_markdown() + '\n')





Name Basics
|    | nconst    | primaryname   |   birthyear |   deathyear | primaryprofession             | knownfortitles                          |
|---:|:----------|:--------------|------------:|------------:|:------------------------------|:----------------------------------------|
|  0 | nm0649057 | Wim Opbrouck  |        1969 |           0 | actor,writer                  | tt0275154,tt5258726,tt3892822,tt0126764 |
|  1 | nm0649059 | Carina Opdam  |           0 |           0 | art_director,costume_designer | tt0138207                               |
|  2 | nm0649060 | Travis Opdyke |           0 |           0 | actor                         | tt0165833                               |
|  3 | nm0649061 | Opec          |           0 |           0 | actress                       | tt0181377                               |
|  4 | nm0649062 | Nancy Opel    |           0 |           0 | actress,costume_designer      | tt1515193,tt0203259,tt0266747,tt0119349 |

Title Basics
|    | tconst  

## Query some basic information from the database

In [ ]:
print('Is Kate Winslet in the database?')
sql = """
SELECT TOP(10) *
FROM nameBasics 
WHERE primaryName LIKE 'Bruce Scivally'
"""
pd.read_sql_query(sql, conn)

In [ ]:
print('Is Morgan Freeman in the database?')
sql = """
SELECT TOP(10) *
FROM nameBasics
WHERE primaryName LIKE '%%'
"""

pd.read_sql_query(sql, conn).head()


In [ ]:
print('Is King of Queens in the database?')
sql = """
SELECT TOP(1) tconst, titleType, primaryTitle
FROM titleBasics
WHERE primaryTitle LIKE '%king of queens%'
"""

pd.read_sql_query(sql, conn).head()

In [ ]:
print('Are there any duplicate actors/actresses in the database?')
sql="""
SELECT DISTINCT primaryName, count(primaryName) AS countActor
FROM nameBasics
WHERE primaryProfession LIKE '%actor%' OR
	  primaryProfession LIKE '%actress%'
GROUP BY primaryName ORDER BY countActor DESC
"""
print(pd.read_sql_query(sql, conn).head(50).to_markdown())

In [ ]:
print("What is the longest running show on television? sort by seasonNumber and then episode number")
sql="""
SELECT TOP(5) tB.primaryTitle, MAX(tE.seasonNumber) as SN, MAX(tE.episodeNumber) as EN,  
    MAX(tE.episodeNumber)*MAX(tE.seasonNumber) AS EpCount
FROM titleEpisode AS tE INNER JOIN titleBasics as tB ON tE.parentTconst=tB.tconst
GROUP BY tB.primaryTitle
HAVING MAX(tE.episodeNumber)*MAX(tE.seasonNumber) > 8000
ORDER BY EpCount DESC
"""
pd.read_sql_query(sql, conn).head()


In [ ]:
print('IMDB top 50')

sql="""
SELECT tb.primaryTitle, tr.averageRating, tr.numVotes, tb.titleType
FROM titleRatings as tr JOIN titleBasics as tb on tr.tconst = tb.tconst
WHERE tr.numVotes > 50000 and tb.titleType LIKE 'movie'
ORDER BY CAST(tr.averageRating AS FLOAT) DESC, CAST(tr.numVotes AS INT) DESC
"""

print(pd.read_sql_query(sql, conn).head(50).to_markdown())

## Create actor-actor network

In [4]:
print('Which two actors have worked together the most')
print('Takes 25 minutes to execute All rows from CTE0 (~7million total) yields 21,565,150 Rows')
sql="""
    With 
    CTE0 as (
        SELECT tp.tconst, tp.nconst 
        FROM titlePrincipals as tp JOIN titleBasics as tb on tb.tconst = tp.tconst
        WHERE (tb.titleType LIKE 'movie' OR tb.titleType LIKE 'tvMovie' OR tb.titleType LIKE 'video') 
        AND tb.startYear > 1950 AND tb.isAdult = 0 AND tb.runtimeMinutes > 20
    ),
	CTE1 as (
		SELECT tp.tconst as tconst, nb.primaryName as primaryName
		FROM CTE0 as tp JOIN nameBasics as nb on nb.nconst = tp.nconst
	),
	CTE2 AS (
		SELECT c1.primaryName as name1, c2.primaryName as name2
		FROM CTE1 as c1 JOIN CTE1 as c2 on c1.tconst=c2.tconst
		WHERE c1.primaryName != c2.primaryName AND c1.primaryName < c2.primaryName 
	)
	SELECT name1, name2, COUNT(name1) as num_appearances
	FROM CTE2
	GROUP BY name1, name2
    ORDER BY num_appearances DESC
    """

df_final = pd.read_sql_query(sql, conn)
print(df_final.head(50).to_markdown())
# df_final.to_csv('df_final.csv')

Which two actors have worked together the most
Takes 25 minutes to execute All rows from CTE0 (~7million total) yields 21,565,150 Rows
|    | name1                        | name2                      |   num_appearances |
|---:|:-----------------------------|:---------------------------|------------------:|
|  0 | Laxmikant Shantaram Kudalkar | Pyarelal Ramprasad Sharma  |               248 |
|  1 | Veronique Del Rosario-Corpus | Vincent Del Rosario III    |               200 |
|  2 | Jacob Grimm                  | Wilhelm Grimm              |               195 |
|  3 | ShÃ´ji Sakai                 | Yutaka Ikejima             |               176 |
|  4 | Kevin Dunn                   | Vince McMahon              |               163 |
|  5 | Kuninori Yamazaki            | Sachi Hamano               |               146 |
|  6 | Hsing-Lung Chiang            | Runme Shaw                 |               132 |
|  7 | Chris Ridenhour              | David Michael Latt         |               1

In [ ]:
df_final = pd.read_csv('df_final.csv', index_col=[0])

In [263]:
df = df_final.copy()
df.columns = ['source', 'target', 'weight']

In [264]:
df = df[df['weight'] > 2.5]

In [265]:
# keep only names that appear in the list 20 times 
dftemp = pd.concat([df['source'], df['target']], ignore_index=True).value_counts()
jkeep = 5
node_labels = dftemp[(dftemp > jkeep)].index

num_node_labels = len(node_labels)
node_encoding = list(range(num_node_labels))

map_key = {node_labels[j]:node_encoding[j] for j in range(num_node_labels)}
df['source'] = df['source'].map(map_key)
df['target'] = df['target'].map(map_key)

df = df.dropna()

In [271]:
with open("hollywood_celebs.txt", "rb") as f:
    hollywood_celebs = pickle.load(f)
hollywood_celebs = hollywood_celebs[0:200]

In [272]:
actor_name = [map_key[j] for j in hollywood_celebs if j in list(map_key.keys())]
df = df[df['source'].isin(actor_name) | df['target'].isin(actor_name)]

In [273]:
edges = [(df.iloc[j]['source'], df.iloc[j]['target'], df.iloc[j]['weight']) for j in range(len(df)) if df.iloc[j]['weight']>1]

In [274]:
unique_nodes = list(np.unique(list(df['source'].astype(int)) + list(df['target'].astype(int))))
# compute size of nodes based on number of connections
node_size = []
for jactor in unique_nodes:
    actor_weight = df[(df['target'] == jactor) | (df['source'] == jactor)]['weight']
    node_size.append(np.sum(actor_weight))
node_size = 150*np.array(node_size)/np.max(node_size)

In [279]:
net = Network(notebook=True, cdn_resources='remote',
              bgcolor='#222222', font_color='white', height='500px',
              width='100%', select_menu=True, filter_menu=True)
net.add_nodes(unique_nodes, label=node_labels[unique_nodes], size=node_size)

net.add_nodes(unique_nodes, size=node_size)
net.add_edges(edges)
net.repulsion(node_distance=200, spring_length=10)
net.show_buttons()
net.show("hollywood_network.html");

print(len(unique_nodes), len(edges))

hollywood_network.html
939 1513


## Create Gradio app

In [261]:
with open("spaces_upload\\unique_nodes.txt", "wb") as f:
    pickle.dump(unique_nodes, f)
with open("spaces_upload\\node_labels.txt", "wb") as f:
    pickle.dump(node_labels, f)
with open("spaces_upload\\edges.txt", "wb") as f:
    pickle.dump(edges, f)
df.to_csv('spaces_upload\\df_out.csv')

In [184]:
import gradio as gr

def needs_analysis():
    # compute size of nodes based on number of connections
    node_size = []
    for jactor in unique_nodes:
        actor_weight = df[(df['target'] == jactor) | (df['source'] == jactor)]['weight']
        node_size.append(np.sum(actor_weight))
    node_size = 50*np.array(node_size)/np.max(node_size)

    net = Network(notebook=True,cdn_resources='remote',bgcolor='#222222',font_color='white',height='750px',width='100%',select_menu=True,filter_menu=True)
    net.add_nodes(unique_nodes, label=node_labels[unique_nodes], size=node_size)

    net.add_edges(edges)
    net.repulsion(node_distance=200, spring_length=10)
    # net.show_buttons()
    net.toggle_physics(False)
    html = net.generate_html()
    #need to remove ' from HTML
    html = html.replace("'", "\"")
    
    return f"""<iframe style="width: 100%; height: 600px;margin:0 auto" name="result" allow="midi; geolocation; microphone; camera; 
    display-capture; encrypted-media;" sandbox="allow-modals allow-forms 
    allow-scripts allow-same-origin allow-popups 
    allow-top-navigation-by-user-activation allow-downloads" allowfullscreen="" 
    allowpaymentrequest="" frameborder="0" srcdoc='{html}'></iframe>"""

with gr.Blocks() as demo:
    gr.HTML(value=needs_analysis())

# demo = gr.Interface(
#     needs_analysis,
#     inputs=None,
#     outputs=gr.outputs.HTML(),
#     title="pyvis_in_gradio",
#     allow_flagging='never'
# )

demo.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
